In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
X = dataset.iloc[:, 3:13].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [20]:
y = dataset.iloc[:, 13].values
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
labelencoder_X = LabelEncoder()  
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X

array([[228, 'France', 'Female', ..., 1, 1, 101348.88],
       [217, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [111, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [318, 'France', 'Female', ..., 0, 1, 42085.58],
       [381, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [401, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [22]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
labelencoder_X_1 = LabelEncoder()  
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
X

array([[228, 0, 'Female', ..., 1, 1, 101348.88],
       [217, 2, 'Female', ..., 0, 1, 112542.58],
       [111, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [318, 0, 'Female', ..., 0, 1, 42085.58],
       [381, 1, 'Male', ..., 1, 0, 92888.52],
       [401, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [23]:
labelencoder_X_2 = LabelEncoder()  
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
X

array([[228, 0, 0, ..., 1, 1, 101348.88],
       [217, 2, 0, ..., 0, 1, 112542.58],
       [111, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [318, 0, 0, ..., 0, 1, 42085.58],
       [381, 1, 1, ..., 1, 0, 92888.52],
       [401, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

Since there is no relational order between the categories of our categorical variable, so for that we need to create a dummy variable for the country categorical variable as it contains three categories unlike the gender variable having only two categories, which is why we will be removing one column to avoid the dummy variable trap. It is useless to create the dummy variable for the gender variable. We will use the OneHotEncoder class to create the dummy variables.

In [24]:
from sklearn.compose import ColumnTransformer  
label_encoder_x_1 = LabelEncoder()  
X[: , 2] = label_encoder_x_1.fit_transform(X[:,2])  
X

array([[228, 0, 0, ..., 1, 1, 101348.88],
       [217, 2, 0, ..., 0, 1, 112542.58],
       [111, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [318, 0, 0, ..., 0, 1, 42085.58],
       [381, 1, 1, ..., 1, 0, 92888.52],
       [401, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [25]:
transformer = ColumnTransformer(
    transformers=[  
        ("OneHot",        # Just a name  
         OneHotEncoder(), # The transformer class  
         [1]              # The column(s) to be applied on.  
         )  
    ],  
    remainder='passthrough' # don't apply anything to the remaining columns   
)  
X = transformer.fit_transform(X.tolist())  
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [26]:
X.shape

(10000, 12)

By having a look at X, we can see that all the columns are of the same type now. Also, the type is no longer an object but float64. We can see that we have twelve independent variables because we have three new dummy variables.

Next, we will remove one dummy variable to avoid falling into a dummy variable trap. We will take a matrix of features X and update it by taking all the lines of this matrix and all the columns except the first one.

In [27]:
X = X[:, 1:]
X.shape

(10000, 11)

It can be seen that we are left with only two dummy variables, so no more dummy variable trap.

Now we are ready to split the dataset into the training set and test set. We have taken the test size to 0.2 for training the ANN on 8,000 observations and testing its performance on 2,000 observations.

In [28]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)  

Besides parallel computations, we are going to have highly computed intensive calculations as well as we don't want one independent variable dominating the other one, so we will be applying feature scaling to ease out all the calculations.

In [29]:
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

### Building an ANN

In [30]:
import keras
from keras.models import Sequential  
from keras.layers import Dense  

After importing the Keras library, we will now import two modules, i.e., the Sequential module, which is required to initialize our neural network, and the Dense module that is needed to build the layer of our ANN.

Next, we will initialize the ANN, or simply we can say we will be defining it as a sequence of layers. The deep learning model can be initialized in two ways, either by defining the sequence of layers or defining a graph. Since we are going to make our ANN with successive layers, so we will initialize our deep learning model by defining it as a sequence of layers.

It can be done by creating an object of the sequential class, which is taken from the sequential model. The object that we are going to create is nothing but the model itself, i.e., a neural network that will have a row of classifiers because we are solving a classification problem where we have to predict a class, so our neural network model is going to be a classifier. As in the next step, we will be predicting the test set result using the classifier name, so we will call our model as a classifier that is nothing but our future Artificial Neural Network that we are going to build.

Since this classifier is an object of Sequential class, so we will be using it, but will not pass any argument because we will be defining the layers step by step by starting with the input layer followed by adding some hidden layers and then the output layer.

In [32]:
classifier = Sequential()

After this, we will start by adding the input layer and the first hidden layer. We will take the classifier that we initialized in the previous step by creating an object of the sequential class, and we will use the add() method to add different layers in our neural network. In the add(), we will pass the layer argument, and since we are going to add two layers, i.e., the input and first hidden layer, which we will be doing with the help of Dense() function that we have mentioned above.

Within the Dense() function we will pass the following arguments;

* units are the very first argument, which can be defined as the number of nodes that we want to add in the hidden layer.
* The second argument is the kernel_initializer that randomly initializes the weight as a small number close to zero so that they can be randomly initialized with a uniform function. Here we have a simple uniform function that will initialize the weight according to the uniform distribution.
* The third argument is the activation, which can be understood as the function that we want to choose in our hidden layer. So, we will be using the rectifier function for the hidden layers and the sigmoid function for the output layer. Since we are in the hidden layer, we are using the "relu" perimeter as it corresponds to the rectifier function.
* And the last is the input_dim argument that specifies the number of nodes in the input layer, which is actually the number of independent variables. It is very necessary to add the argument because, by so far, we have only initialized our ANN, we haven't created any layer yet, and that's why it doesn't know which node this hidden layer we are creating is expecting as inputs. After the first hidden layer gets created, we don't need to specify this argument for the next hidden layers.

In [33]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

Next, we will add the second hidden layer by using the same add method followed by passing the same parameter, which is the Dense() as well as the same parameters inside it as we did in the earlier step except for the input_dim.

In [34]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

After adding the two hidden layers, we will now add the final output layer. This is again similar to the previous step, just the fact that we will be units parameter because in the output layer we only require one node as our dependent variable is a categorical variable encompassing a binary outcome and also when we have binary outcome then, in that case, we have only one node in the output layer. So, therefore, we will put units equals to 1, and since we are in the output layer, we will be replacing the rectifier function to sigmoid activation function.

In [35]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))  

As we are done with adding the layers of our ANN, we will now compile the whole artificial neural network by applying the stochastic gradient descent. We will start with our classifier object, followed by using the compile method and will pass on the following arguments in it.

* The first argument is the optimizer, which is simply the algorithm that we want to use to find the optimal set of weights in the neural networks. The algorithm that we are going to use is nothing but the stochastic gradient descent algorithm. Since there are several types of stochastic descent algorithms and the most efficient one is called "adam," which is going to be the input of this optimizer parameter.
* The second parameter is the loss, which is a loss function within the stochastic gradient descent algorithm, which is used to find the optimal weights. Since our dependent variable has a binary outcome, so we will be using binary_crossentropy logarithmic function, and when there is a binary outcome, then we will incorporate categorical_crossentropy.
* The last argument will be the metrics, which is nothing but a criterion to evaluate our model, and we are using the "accuracy." So, what happens is when the weights are updated after each observation, the algorithm makes use of this accuracy to improve the model's performance.

In [36]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])  

Next, we will fit the ANN to the training set for which we will be using the fit method to fit our ANN to the training set. In the fit method, we will be passing the following arguments:

* The first argument is the dataset on which we want to train our classifier, which is the training set separated into two-argument such as X_train (matrix of feature containing the observations of the train set) and y_train (containing the actual outcomes of the dependent variable for all the observations in the training set).
* The next argument is the batch_size, which is the number of observations, after which we want to update the weight.
* And lastly, the no. of epochs that we are going to apply to see the algorithm in action as well the improvement in accuracy over the different epochs.

In [37]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)  

Epoch 1/100
800/800 [==============================] - 5s 4ms/step - loss: 0.4927 - accuracy: 0.7952
Epoch 2/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4153 - accuracy: 0.8109
Epoch 3/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3979 - accuracy: 0.8281
Epoch 4/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3864 - accuracy: 0.8303
Epoch 5/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3786 - accuracy: 0.8363
Epoch 6/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3730 - accuracy: 0.8440
Epoch 7/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3683 - accuracy: 0.8479
Epoch 8/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3648 - accuracy: 0.8515
Epoch 9/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3614 - accuracy: 0.8520
Epoch 10/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3594 - accura

800/800 [==============================] - 3s 4ms/step - loss: 0.3355 - accuracy: 0.8620
Epoch 82/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3360 - accuracy: 0.8633
Epoch 83/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3359 - accuracy: 0.8633
Epoch 84/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3362 - accuracy: 0.8626
Epoch 85/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3362 - accuracy: 0.8615
Epoch 86/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3363 - accuracy: 0.8612
Epoch 87/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3358 - accuracy: 0.8620
Epoch 88/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3357 - accuracy: 0.8616
Epoch 89/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3356 - accuracy: 0.8620
Epoch 90/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3355 - accuracy: 

From the output image given above, you can see that our model is ready and has reached an accuracy of 84% approximately

### Making the Predictions and Evaluating the Model

In [52]:
y_pred = classifier.predict(X_test)
y_pred

63/63 [==============================] - 0s 3ms/step


array([[0.29214492],
       [0.26754275],
       [0.0613675 ],
       ...,
       [0.15588614],
       [0.10529058],
       [0.2737743 ]], dtype=float32)

From the output image given above, we can see all the probabilities that the 2,000 customers of the test set will leave the bank. For example, if we have a look at first probability, i.e., 25% means that this first customer of the test set, indexed by zero, has a 25% chance to leave the bank.

Since the predicted method returns the probability of the customers leave the bank and in order to use this confusion matrix, we don't need these probabilities, but we do need the predicted results in the form of True or False. So, we need to transform these probabilities into the predicted result.

We will choose a threshold value to decide when the predicted result is one, and when it is zero. So, we predict 1 over the threshold and 0 below the threshold as well as the natural threshold that we will take is 0.5, i.e., 50%. If the y_pred is larger, then it will return True else False.

In [53]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [54]:
from sklearn.metrics import confusion_matrix  
cm = confusion_matrix(y_test, y_pred)  
cm

array([[1553,   42],
       [ 222,  183]], dtype=int64)